In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import time
!pip install torchsummary

from torchsummary import summary
from tqdm import tqdm
import tensorflow
import tensorflow as tf
from sklearn.metrics import accuracy_score
from torchvision import models




In [4]:
# Implementation of Alexnet

class Conv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super(Conv, self).__init__()
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size,
                              stride=stride, padding=padding)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.lrn = nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2)

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.lrn(x)
        return x

class AlexNetSmall(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNetSmall, self).__init__()
        self.conv1 = Conv(in_channels=3, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.conv2 = Conv(in_channels=64, out_channels=64, kernel_size=5, stride=1, padding=2)
        # two fully connected layers
        self.fc1 = nn.Linear(64 * 6 * 6, 384)
        self.fc2 = nn.Linear(384, 192)
        # 10-way linear layer is used for prediction
        self.fc3 = nn.Linear(192, num_classes)
        self.relu = nn.ReLU()

              



    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x
    

model = AlexNetSmall()
summary(model,(3,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 28, 28]           4,864
              ReLU-2           [-1, 64, 28, 28]               0
         MaxPool2d-3           [-1, 64, 13, 13]               0
 LocalResponseNorm-4           [-1, 64, 13, 13]               0
              Conv-5           [-1, 64, 13, 13]               0
            Conv2d-6           [-1, 64, 13, 13]         102,464
              ReLU-7           [-1, 64, 13, 13]               0
         MaxPool2d-8             [-1, 64, 6, 6]               0
 LocalResponseNorm-9             [-1, 64, 6, 6]               0
             Conv-10             [-1, 64, 6, 6]               0
           Linear-11                  [-1, 384]         885,120
             ReLU-12                  [-1, 384]               0
           Linear-13                  [-1, 192]          73,920
             ReLU-14                  [

In [5]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameter
batch_size = 64
#  CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(),transforms.CenterCrop(28),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.CIFAR10(root='./', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./', train=False, transform=transform)



train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar-10-python.tar.gz to ./


In [7]:
def LabelCorruption(percentage,myloader):
    labels = myloader.dataset.targets[:]
    data = myloader.dataset.data[:]
    num = int(len(labels)*percentage)
    idxes = []
    print(num)
    # generate unique random index
    while len(idxes) < num:
        idx = np.random.randint(0,len(labels))
        if idx not in idxes:
            idxes.append(idx)
    # change the label of the index
    for idx in idxes:
        randomLabel = np.random.randint(0,10)
        while randomLabel == labels[idx]:
            randomLabel = np.random.randint(0,10)
        labels[idx] = randomLabel
    
    corrupted_dataset = torch.utils.data.TensorDataset(torch.Tensor(data),torch.Tensor(labels).long())
    corrupted_dataset.data = data
    corrupted_dataset.targets = labels

    corrupted_loader = torch.utils.data.DataLoader(dataset=corrupted_dataset, batch_size=batch_size, shuffle=True)


    return corrupted_loader



In [10]:
def trainmodel(epochs=10,train_loader=train_loader,test_loader=test_loader,lr=0.001,corruption=0):
    model = AlexNetSmall()
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    train_loss = []
    test_loss = []
    train_acc = []
    test_acc = []
    if corruption>0:
        train_loader = LabelCorruption(percentage=corruption,train_loader=train_loader)
    time1 = time.time()
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        running_acc = 0.0
        for i, (images, labels) in enumerate(tqdm(train_loader)):
            images = images.to(device)
            labels = labels.to(device)
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # Compute accuracy
            _, argmax = torch.max(outputs, 1)
            accuracy = (labels == argmax.squeeze()).float().mean()
            running_acc += accuracy / len(train_loader)
            running_loss += loss / len(train_loader)
        model.eval()
        with torch.no_grad():
            test_loss = 0.0
            test_acc = 0.0
            for images, labels in test_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                _, argmax = torch.max(outputs, 1)
                accuracy =  accuracy_score(labels.to(device) , argmax.to(device))
                test_acc += accuracy / len(test_loader)
                test_loss += loss / len(test_loader)

        train_loss.append(running_loss)
        test_loss.append(float(test_loss))
        train_acc.append(running_acc)
        test_acc.append(test_acc)
        # print("Epoch:" , epoch , "Train Loss" , running_loss , "Test Loss" , test_loss , "Train Accuracy" , running_acc , "Test Accuracy" , test_acc)
    time2 = time.time()
    timetaken = time2-time1
    return train_loss, test_loss, train_acc, test_acc , model,timetaken


In [9]:

correctTrain_loss, correctTest_loss, correctTrain_acc, correctTest_acc , correctmodel,correcttimetaken = trainmodel(epochs=10,train_loader=train_loader,test_loader=test_loader,lr=0.001)

 43%|████▎     | 340/782 [00:52<01:08,  6.47it/s]


KeyboardInterrupt: 

In [9]:
criterion = nn.CrossEntropyLoss()
model = AlexNetSmall()
model.to(device)
with torch.no_grad():
           test_loss = 0.0
           test_acc = 0.0
           for images, labels in test_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                _, argmax = torch.max(outputs, 1)
                accuracy =  accuracy_score(labels.cpu(), argmax.cpu())
                test_acc += accuracy / len(test_loader)
                test_loss += loss / len(test_loader)
                
      

In [14]:
print((test_acc))

0.09982085987261168
